<h1 align=center><b>CSC14120 – PARALLEL PROGRAMMING</b></h1>

<h2 align=center>FINAL PROJECT</h2>
<p style="font-size:32px;text-align:center">Parallelize Convolutional Layer in the LeNet-5 Architecture using CUDA</p>

**Student information**

Student name         | Student ID 
---------------------|-------------
Nguyen Quang Gia Bao | 20120040
Huynh Tuan Nam       | 20120136
Tran Hoang Anh Phi   | 20120158

### __Table of Contents__

1. [Introduction](#introduction)
   1. [Objectives](#objectives)
   2. [Dataset](#dataset)
2. [Background](#background)
   1. [Convolutional Neural Networks (CNNs)](#cnns)
   2. [LeNet-5](#lenet-5)
3. [Starter project](#starter)
4. [Implementation](#implementation)
   1. [Convolutional Layer](#convolutional-layer)
   2. [Pooling Layer](#pooling-layer)
   3. [Fully Connected Layer](#fully-connected-layer)
   4. [LeNet-5 Architecture](#lenet-5-architecture)
   5. [Parallelization](#parallelization)
5. [Usage](#usage)
   1. [Prerequisites](#prerequisites)
   2. [Training](#training)
   3. [Testing](#testing)
6. [Results](#results)
   1. [Training stage](#training-stage)
   2. [Testing stage](#testing-stage)
      1. [Basic kernel](#basic-kernel)
      2. [Optimized kernel 1](#optimized-kernel-1)
      3. [Optimized kernel 2](#optimized-kernel-2)
      4. [Optimized kernel 3](#optimized-kernel-3)
7. [Conclusion](#conclusion)
8. [References](#references)

# 1    Introduction <a id="introduction"></a>

## 1.1    Objectives <a id="objectives"></a>

__Objective__: Implement and optimize the forward-pass of a convolutional layer using CUDA. 
- Convolutional layers are the primary building blocks of convolutional neural networks (CNNs), which are used in many machine learning tasks.
- In general, CNNs work well on tasks where the data/input features have some level of partial relationship.

<center>

![](https://www.datasciencecentral.com/wp-content/uploads/2021/10/1lvvWF48t7cyRWqct13eU0w.jpeg)

</center>

## 1.2    Dataset <a id="dataset"></a>

- __Fashion-MNIST__ is a dataset of Zalando's article images - consisting of a training set of 60,000 examples and a test set of 10,000 examples | Each example is a $28x28$ grayscale image, associated with 10 classes |  Zalando intends Fashion-MNIST to serve as a direct drop-in replacement for the original MNIST dataset for benchmarking machine learning algorithms | It shares the same image size and structure of training and testing splits | 

- Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255. The training and test data sets have 785 columns. The first column consists of the class labels (see above), and represents the article of clothing. The rest of the columns contain the pixel-values of the associated image.
    - To locate a pixel on the image, suppose that we have decomposed x as x = i * 28 + j, where i and j are integers between 0 and 27. The pixel is located on row i and column j of a 28 x 28 matrix.
    - For example, pixel31 indicates the pixel that is in the fourth column from the left, and the second row from the top, as in the ascii-diagram below.


- __Labels__: Each training and test example is assigned to one of the following labels:
<center>

0 | T-shirt/top
--|------------
1 | Trouser
2 | Pullover
3 | Dress
4 | Coat
5 | Sandal
6 | Shirt
7 | Sneaker
8 | Bag
9 | Ankle boot

</center>

# 2. Background <a id="background"></a>

## 2.1    Convolutional Neural Networks (CNNs)<a id="cnns"></a>

- Standford [cheatsheat](https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-convolutional-neural-networks#overview)
  
- Video CNNs: ["How Convolutional Neural Networks work"](https://www.youtube.com/watch?v=FmpDIaiMIeA)

## 2.2    LeNet-5 <a id="lenet-5"></a>

- A Complete Guide: [here](https://www.kaggle.com/code/blurredmachine/lenet-architecture-a-complete-guide/notebook)

# 3. Starter project <a id="starter-project"></a>

# 4. Implementation <a id="implementation"></a>

The original starter project was used the Eigen library version 3.3.4, which is not compatible with CUDA. Therefore, we have to upgrade the Eigen library to version 3.4.0, which is compatible with CUDA. 

## 4.1    Convolutional layer<a id="convolutional-layer"></a>

The convolution layer (CONV) uses filters that perform convolution operations as it is scanning the input $\mathbb{I}$ with respect to its dimensions. Its hyperparameters include the filter size $\mathbb{F}$ and stride $\mathbb{S}$. The resulting output $\mathbb{O}$ is called feature map or activation map.
<center>

![](https://stanford.edu/~shervine/teaching/cs-230/illustrations/architecture-cnn-en.jpeg?3b7fccd728e29dc619e1bd8022bf71cf)

</center> 

The sequential implementation of the convolutional layer was use the im2col method to convert the input image into a matrix and then perform matrix multiplication with the filter matrix to get the output image. The im2col algorithm is a technique that converts an image into a matrix, such that each column of the matrix corresponds to a small patch of the image. This makes it easier to perform convolution operations using matrix multiplication, which can be faster and more efficient than looping over the image pixels. The im2col method is working as follows:
- Convert input image of size O(HWC) to a patches matrix of size O(HW(K^2)C) 
- Convert filter into format __kernel height__ * __kernel width__ * __kernel channel__
- Multiply the modified input and filter matrix using GEMM matrix multiplication to get the output. This is a single call.

To illustrate the im2col algorithm with a greyscale example, let's assume we have a 4x4 image with one channel, and we want to apply a 2x2 filter with a stride of 1 and no padding. The im2col algorithm would produce a 4x9 matrix, where each column represents a 2x2 patch of the image, as shown below:

$$
\begin{bmatrix}
a & b & c & d \\
e & f & g & h \\
i & j & k & l \\
m & n & o & p \\
\end{bmatrix}
\rightarrow
\begin{bmatrix}
a & b & c & e & f & g & i & j & k \\
b & c & d & f & g & h & j & k & l \\
e & f & g & i & j & k & m & n & o \\
f & g & h & j & k & l & n & o & p \\
\end{bmatrix}
$$

Now, we can multiply this matrix by a vectorized version of the filter, and reshape the result into a 3x3 output image. For example, if the filter is:

$$
\begin{bmatrix}
1 & 2 \\
3 & 4 \\
\end{bmatrix}
$$

Then the vectorized filter is:

$$
\begin{bmatrix}
1 \\
2 \\
3 \\
4 \\
\end{bmatrix}
$$

And the output image is:

$$
\begin{bmatrix}
a + 2b + 3e + 4f & b + 2c + 3f + 4g & c + 2d + 3g + 4h \\
e + 2f + 3i + 4j & f + 2g + 3j + 4k & g + 2h + 3k + 4l \\
i + 2j + 3m + 4n & j + 2k + 3n + 4o & k + 2l + 3o + 4p \\
\end{bmatrix}
$$

This is equivalent to sliding the filter over the image and computing the dot product at each position, but it can be done more efficiently using matrix operations. 

Here is the pseudo code for the im2col method:

```c++
input[C][H][W];
kernels[M][K][K][C];
output[M][H][W];
for h in 1 to H do
    for w in 1 to W do
        for o in 1 to M do
            sum = 0;
            for x in 1 to K do
                for y in 1 to K do
                    for i in 1 to C do
                    sum += input[i][h+y][w+x] * kernels[o][x][y][i];
         output[o][w][h] = sum;
```

## 4.2    Pooling layer<a id="pooling-layer"></a>

The pooling layer is a downsampling operation, typically applied after a convolution layer, which does some spatial invariance. There has been a lot of research on the pooling layer, and the most common pooling methods are max pooling and average pooling. In this project, we will implement both of them. 
<p align="center">
    <img src="https://www.mdpi.com/remotesensing/remotesensing-13-04712/article_deploy/html/images/remotesensing-13-04712-g005.png" width="600" height="400">
</p>

<center>

Type | Max pooling | Average pooling
-----|-------------|----------------
Purpose | Each pooling operation selects the maximum value of the current view | Each pooling operation averages the values of the current view
Illustration | ![](https://stanford.edu/~shervine/teaching/cs-230/illustrations/max-pooling-a.png?711b14799d07f9306864695e2713ae07) | ![](https://stanford.edu/~shervine/teaching/cs-230/illustrations/average-pooling-a.png?58f9ab6d61248c3ec8d526ef65763d2f)
Comments | - Preserves detected features <br> - Most commonly used | - Downsamples features map <br> - Used in LeNet

</center>

In our implementation, max pooling was utilized. Max pooling is a pooling operation that selects the maximum element from the region of the feature map covered by the filter. Thus, the output after max pooling will be a feature map containing the most prominent features of the previous feature map. 

This code implements the forward pass of a max pooling operation. Max pooling is a common operation in convolutional neural networks (CNNs) used to reduce the spatial dimensions of an input feature map while retaining the most important information.

Let's go through the workflow of this code step by step:
```bash
# Input:
# bottom: Matrix object representing the input feature map
# dim_out: Number of output channels
# n_sample: Number of samples in the input feature map
# hw_in: Total number of elements in the input feature map
# hw_pool: Total number of elements in the pooling window
# hw_out: Total number of elements in the output feature map

# Output:
# top: Matrix object representing the output feature map
# max_idxs: Vector storing the indices of the maximum values in each pooling window for each sample

# Initialize top matrix with dimensions dim_out x n_sample and set all elements to the lowest possible float value
top = Matrix(dim_out, n_sample)
top.fill(std::numeric_limits<float>::lowest())

# Initialize max_idxs vector
max_idxs = Vector()

# Iterate over each sample in the input feature map
for sample in range(n_sample):
    # Retrieve the image vector for the current sample
    image = bottom.get_sample(sample)
    
    # Iterate over each channel in the input feature map
    for channel in range(dim_out):
        # Iterate over each element in the output feature map
        for i_out in range(hw_out):
            # Calculate the row and column indices of the top-left corner of the pooling window
            step_h = i_out // (hw_out // hw_pool)
            step_w = i_out % (hw_out // hw_pool)
            
            # Calculate the start index of the pooling window in the input feature map
            start_idx = channel * hw_in + step_h * hw_pool * hw_in + step_w * hw_pool
            
            # Iterate over each element in the pooling window
            for i_pool in range(hw_pool):
                # Check if the current element is out of range
                if start_idx + i_pool >= hw_in:
                    continue
                
                # Calculate the pick index of the current element in the input feature map
                pick_idx = start_idx + i_pool
                
                # Compare the value of the current element in the input feature map with the corresponding element in the output feature map
                if image[pick_idx] > top[channel * hw_out + i_out, sample]:
                    # Update the corresponding element in the output feature map
                    top[channel * hw_out + i_out, sample] = image[pick_idx]
                    
                    # Store the pick index in the max_idxs vector
                    max_idxs.push_back(pick_idx)

```

Because the pooling layer dont have any trainable parameters, the backward pass is very simple. We just need to propagate the gradient from the output feature map to the input feature map. 

In summary, this code performs max pooling by iterating over each sample, channel, and element in the output feature map. It calculates the pooling window's top-left corner index, checks if the window is within range, and updates the output feature map with the maximum value in each pooling window. The corresponding indices of the maximum values are stored in the `max_idxs` vector.

In particular, max and average pooling are special kinds of pooling where the maximum and average value is taken, respectively.

## 4.3    Fully connected layer<a id="fully-connected-layer"></a>

The fully connected layer (FC) operates on a flatten input where each input is connected to all neurons.

## 4.4    LeNet-5 architecture<a id="lenet-5-architecture"></a>

Instead of using the original LeNet-5 architecture, we will use a modified version of it. The original LeNet-5 architecture is shown below:
![](https://d2l.ai/_images/lenet.svg)
And the modified version is shown below:

__First Layer: Convolutional Layer (CONV1):__

- Parameters: Input (N) = 28, Padding (P) = 2, Filter (F) = 5 x 5, Stride (S) = 1
- Conv Operation: ((N + 2P - F) / S) + 1 = ((28 + 4 - 5) / 1) + 1 = 28 x 28
- We will apply 6 filters / kernels so we will get a 28 x 28 x 6 dimensional output

__Second Layer: Max Pooling Layer (POOL1):__

- Parameters: Input (N) = 28, Filter (F) = 2 x 2, Stride (S) = 2
- AVG Pooling Operation: ((N + 2P -F) / S) + 1 = ((28 - 2) / 2) + 1 = 14 x 14
- We will have a 14 x 14 x 6 dimensional output at the end of this pooling

__Third Layer: Convolutional Layer (CONV2):__

- Parameters: Input (N) = 14, Filter (F) = 5 x 5, Stride (S) = 1
- Conv Operation: ((N + 2P - F) / S) + 1 = ((14 - 5) / 1) + 1 = 10 x 10
- We will apply 16 filters / kernels so we will get a 10 x 10 x 16 dimensional output

__Fourth Layer: Max Pooling Layer (POOL2):__

- Parameters: Input (N) = 10, Filter (F) = 2 x 2, Stride (S) = 2
- AVG Pooling Operation: ((N + 2P -F) / S) + 1 = ((10 - 2) / 2) + 1 = 5 x 5
- We will have a 5 x 5 x 16 dimensional output at the end of this pooling

__Fifth Layer: Fully Connected layer(FC1):__

- Parameters: W: 400 * 120, b: 120
- We will have an output of 120 x 1 dimension

__Sixth Layer: Fully Connected layer(FC2):__

- Parameters: W: 120 * 84, b: 84
- We will have an output of 84 x 1 dimension

__Seventh Layer: Output layer(Softmax):__

- Parameters: W: 84 * 10, b: 10
- We will get an output of 10 x 1 dimension

```python
model = Sequential()
model.add(Conv2D(filters=6, kernel_size=(5,5), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPool2D(strides=2))
model.add(Conv2D(filters=16, kernel_size=(5,5), padding='valid', activation='relu'))
model.add(MaxPool2D(strides=2))
model.add(Flatten())
model.add(Dense(120, activation='relu'))
model.add(Dense(84, activation='relu'))
model.add(Dense(10, activation='softmax'))
```

## 4.5    Parallelize<a id="parallelize"></a>

According to the teacher's instructions, we will only parallelize the __forward-pass__ of the convolutional layer using CUDA. 


### 4.5.1    Basic kernel<a id="basic-kernel"></a>

```cuda-cpp
__global__ void conv_forward_kernel(float *d_output, const float *d_input, const float *d_weight,
                                    const int n_sample, const int channel_out, const int channel_in,
                                    const int height_in, const int width_in, const int height_kernel)
```
This line defines a CUDA kernel function called `conv_forward_kernel`. It takes several input arguments, including pointers to `d_output`, `d_input`, and `d_weight` arrays, which represent the output, input, and weight tensors, respectively. The other arguments represent the dimensions of the tensors.

```cuda-cpp
const int height_out = height_in - height_kernel + 1;
const int width_out = width_in - height_kernel + 1;
```
These two lines calculate the height and width of the output tensor based on the input tensor dimensions and the size of the convolutional kernel.

```cuda-cpp
#define y4d(i3, i2, i1, i0) d_output[(i3) * (channel_out * height_out * width_out) + (i2) * (height_out * width_out) + (i1) * (width_out) + i0]
#define x4d(i3, i2, i1, i0) d_input[(i3) * (channel_in * height_in * width_in) + (i2) * (height_in * width_in) + (i1) * (width_in) + i0]
#define k4d(i3, i2, i1, i0) d_weight[(i3) * (channel_in * height_kernel * height_kernel) + (i2) * (height_kernel * height_kernel) + (i1) * (height_kernel) + i0]
```
These lines define three macros: `y4d`, `x4d`, and `k4d`. These macros are used to access elements of the `d_output`, `d_input`, and `d_weight` arrays, respectively. They provide a convenient way to access elements in a 4-dimensional tensor using indices `i3`, `i2`, `i1`, and `i0`.

```cuda-cpp
int height_grid = ceil(1.0*height_out / TILE_WIDTH);
int width_grid = ceil(1.0*width_out / TILE_WIDTH); 
```
These two lines calculate the number of thread blocks needed to cover the height and width of the output tensor. The `TILE_WIDTH` is a constant that determines the size of each thread block.

```cuda-cpp
int b = blockIdx.x;                 // batch number
int m = blockIdx.y;                 // output feature
int h = (blockIdx.z / width_grid) * TILE_WIDTH + threadIdx.y; // row of the image matrix
int w = (blockIdx.z % width_grid) * TILE_WIDTH + threadIdx.x; // col of the image matrix
```
These lines calculate the indices `b`, `m`, `h`, and `w` for the current thread. `blockIdx` represents the index of the current thread block, and `threadIdx` represents the index of the current thread within the block. These indices are used to determine the position of the current thread in the output tensor.

```cuda-cpp
float accum = 0.0f;
```
This line initializes a variable `accum` to store the accumulated sum of the convolution operation.

```cuda-cpp
if (h < height_out && w < width_out) 
{
    for(int c=0; c<channel_in; c++)             // sum over all input features
    {
        for(int p=0; p<height_kernel; p++)         // KxK filter 
            for(int q=0; q<height_kernel; q++)
                accum += x4d(b, c, h+p, w+q) * k4d(m, c, p, q); // 4 dimensions macro resolve thread index
    }
    y4d(b,m,h,w) = accum;
} // endif (h < H_out && w < W_out)
```
This block of code performs the convolution operation. It checks if the current thread is within the valid range of the output tensor. If so, it iterates over the input channels (`channel_in`), the height and width of the convolutional kernel (`height_kernel`), and accumulates the product of the corresponding input and weight values into the `accum` variable. Finally, it stores the accumulated value in the output tensor using the `y4d` macro.

```cuda-cpp
#undef y4d
#undef x4d
#undef k4d
```
These lines undefine the previously defined macros to avoid any potential conflicts with other parts of the code.

## 4.6    Save and load model<a id="save-load"></a>

In order to test the model, we need to save and load the weights of the model. We implemented the `save_parameters` and `load_parameters` methods in the `Network` class to save and load the weights of the model with binary files. 

# 5. Usage <a id="usage"></a>

## 5.1 Prerequisites <a id="prerequisites"></a>

- Download and unzip [FASHION-MNIST](https://www.kaggle.com/datasets/zalando-research/fashionmnist) dataset in `mini-dnn-cpp/data/fashion-mnist/`.

- Download and unzip [Eigen 3.4.0](https://gitlab.com/libeigen/eigen/-/releases/3.4.0), then place folder __Eigen__ in `mini-dnn-cpp/`.

## 5.2 Training <a id="training"></a>

In [ ]:
!make setup
!make train
!make train_model

## 5.3 Testing <a id="testing"></a>

Please run the following command after each test to clean up the temporary files:

```bash
make clean
```

### 5.3.1 Basic kernel <a id="basic-kernel"></a>

In [10]:
!make clean
!make setup
!make gpu_basic
!make test
!make run

To make your changes take effect please reactivate your environment
rm -f src/layer/*.o
rm test.o
rm test
To make your changes take effect please reactivate your environment
make network.o
make[1]: Entering directory '/mnt/net/i2x256-ai03/hotel/phit/personal/ParallelProgramming/mini-dnn-cpp'
nvcc --compile src/network.cc -o src/network.o -I./ -L/usr/local/cuda/lib64 -lcudart
make[1]: Leaving directory '/mnt/net/i2x256-ai03/hotel/phit/personal/ParallelProgramming/mini-dnn-cpp'
make mnist.o
make[1]: Entering directory '/mnt/net/i2x256-ai03/hotel/phit/personal/ParallelProgramming/mini-dnn-cpp'
nvcc --compile src/mnist.cc -o src/mnist.o  -I./ -L/usr/local/cuda/lib64 -lcudart
make[1]: Leaving directory '/mnt/net/i2x256-ai03/hotel/phit/personal/ParallelProgramming/mini-dnn-cpp'
make layer
make[1]: Entering directory '/mnt/net/i2x256-ai03/hotel/phit/personal/ParallelProgramming/mini-dnn-cpp'
nvcc --compile src/layer/ave_pooling.cc -o src/layer/ave_pooling.o -I./ -L/usr/local/cuda/lib64 -lcuda

### 5.3.2 Optimized kernel 1 <a id="optimize-kernel-1"></a>

In [11]:
!make clean
!make setup
!make gpu_v1
!make test
!make run

To make your changes take effect please reactivate your environment
rm -f src/layer/*.o
rm test.o
rm test
To make your changes take effect please reactivate your environment
make network.o
make[1]: Entering directory '/mnt/net/i2x256-ai03/hotel/phit/personal/ParallelProgramming/mini-dnn-cpp'
nvcc --compile src/network.cc -o src/network.o -I./ -L/usr/local/cuda/lib64 -lcudart
make[1]: Leaving directory '/mnt/net/i2x256-ai03/hotel/phit/personal/ParallelProgramming/mini-dnn-cpp'
make mnist.o
make[1]: Entering directory '/mnt/net/i2x256-ai03/hotel/phit/personal/ParallelProgramming/mini-dnn-cpp'
nvcc --compile src/mnist.cc -o src/mnist.o  -I./ -L/usr/local/cuda/lib64 -lcudart
make[1]: Leaving directory '/mnt/net/i2x256-ai03/hotel/phit/personal/ParallelProgramming/mini-dnn-cpp'
make layer
make[1]: Entering directory '/mnt/net/i2x256-ai03/hotel/phit/personal/ParallelProgramming/mini-dnn-cpp'
nvcc --compile src/layer/ave_pooling.cc -o src/layer/ave_pooling.o -I./ -L/usr/local/cuda/lib64 -lcuda

### 5.3.3 Optimized kernel 2 <a id="optimized-kernel-2"></a>

In [ ]:
!make clean
!make setup
!make gpu_v2
!make test
!make run

### 5.3.4 Optimized kernel 3 <a id="optimized-kernel-3"></a>

In [ ]:
!make clean
!make setup
!make gpu_v3
!make test
!make run

# 6. Results <a id="results"></a>

### Training Stage

```c++
  Network dnn;
  Layer *conv1 = new Conv(1, 28, 28, 6, 5, 5);
  Layer *pool1 = new MaxPooling(6, 24, 24, 2, 2, 2);
  Layer *conv2 = new Conv(6, 12, 12, 16, 5, 5);
  Layer *pool2 = new MaxPooling(16, 8, 8, 2, 2, 2);
  Layer* fc3 = new FullyConnected(pool2->output_dim(), 120);
  Layer* fc4 = new FullyConnected(120, 84);
  Layer* fc5 = new FullyConnected(84, 10);

  Layer* relu1 = new ReLU;
  Layer* relu2 = new ReLU;
  Layer* relu3 = new ReLU;
  Layer* relu4 = new ReLU;
  Layer* relu5 = new ReLU;
  Layer* softmax = new Softmax;
  dnn.add_layer(conv1);
  dnn.add_layer(relu1);
  dnn.add_layer(pool1);
  dnn.add_layer(conv2);
  dnn.add_layer(relu2);
  dnn.add_layer(pool2);
  dnn.add_layer(fc3);
  dnn.add_layer(relu3);
  dnn.add_layer(fc4);
  dnn.add_layer(relu4);
  dnn.add_layer(fc5);
  // dnn.add_layer(relu5);
  dnn.add_layer(softmax);
```

Default hyperparameters:

```c++
  int batch_size = 128;
  float learning_rate = 0.001;
```

1. Train on Fashion-MNIST with 10 epochs
```
weight-bad-1.bin
10-th epoch, test acc: 0.6136
```

2. Train on Fashion-MNIST with 10 epochs (dont use Relu activation at the (n - 1) layer (before softmax layer))
```
weight-1.bin
10-th epoch, test acc: 0.8619
```


3. Train on Fashion-MNIST with 10 epochs with kernel size = 2
```
weight-bad-2.bin
10-th epoch, test acc: 0.625
```

4. Train on Fashion-MNIST with 10 epochs with kernel size = 2 and dont use Relu at the penultimate layer
```
weight-2.bin
10-th epoch, test acc: 0.8766
```  

5. Train on Fashion-MNIST with 15 epochs with kernel size = 2 and dont use Relu at the penultimate layer
```
weight-3.bin
15-th epoch, test acc: 0.8862
```

6. Train on Fashion-MNIST with 20 epochs with kernel size = 2, lr=0.0005 and dont use Relu at the penultimate layer


### Testing Stage

# 7. References <a id="references"></a>

- https://www.kaggle.com/datasets/zalando-research/fashionmnist